In [79]:
#dependecies
import pandas as pd
import numpy as np
import os

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [81]:
df = pd.read_csv(os.path.join('..', "arun/Resources/", 'diagnosis-of-covid-19-and-its-clinical-spectrum.csv'))

In [125]:
#Data pre-processing
data = df[['sars_cov_2_exam_result','patient_age_quantile', 'leukocytes', 'platelets', 'monocytes', 'hematocrit', 'eosinophils', 'red_blood_cells', 'hemoglobin', 'lymphocytes', 'mean_platelet_volume']]

In [126]:
feature_names = ['patient_age_quantile', 'leukocytes', 'platelets', 'monocytes', 'hematocrit', 'eosinophils', 'red_blood_cells', 'hemoglobin', 'lymphocytes', 'mean_platelet_volume']

In [127]:
y = data['sars_cov_2_exam_result'].values
X = data.drop('sars_cov_2_exam_result', axis=1).values
print(X.shape, y.shape)

(5644, 10) (5644,)


In [128]:
#handle missing data
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, 1:len(feature_names)])
X[:, 1:len(feature_names)] = imputer.transform(X[:, 1:len(feature_names)])

In [129]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [130]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [131]:
#Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [132]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [133]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [138]:
# Creating model and adding layers 
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=10))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [139]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [140]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 100)               1100      
_________________________________________________________________
dense_22 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 202       
Total params: 11,402
Trainable params: 11,402
Non-trainable params: 0
_________________________________________________________________


In [141]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Train on 4233 samples
Epoch 1/60
4233/4233 - 0s - loss: 0.3501 - accuracy: 0.8989
Epoch 2/60
4233/4233 - 0s - loss: 0.3210 - accuracy: 0.8991
Epoch 3/60
4233/4233 - 0s - loss: 0.3198 - accuracy: 0.8991
Epoch 4/60
4233/4233 - 0s - loss: 0.3170 - accuracy: 0.8991
Epoch 5/60
4233/4233 - 0s - loss: 0.3139 - accuracy: 0.8991
Epoch 6/60
4233/4233 - 0s - loss: 0.3111 - accuracy: 0.8991
Epoch 7/60
4233/4233 - 0s - loss: 0.3126 - accuracy: 0.8991
Epoch 8/60
4233/4233 - 0s - loss: 0.3079 - accuracy: 0.8991
Epoch 9/60
4233/4233 - 0s - loss: 0.3085 - accuracy: 0.8991
Epoch 10/60
4233/4233 - 0s - loss: 0.3071 - accuracy: 0.9001
Epoch 11/60
4233/4233 - 0s - loss: 0.3075 - accuracy: 0.8994
Epoch 12/60
4233/4233 - 0s - loss: 0.3078 - accuracy: 0.9001
Epoch 13/60
4233/4233 - 0s - loss: 0.3046 - accuracy: 0.9005
Epoch 14/60
4233/4233 - 0s - loss: 0.3054 - accuracy: 0.9008
Epoch 15/60
4233/4233 - 0s - loss: 0.3044 - accuracy: 0.9017
Epoch 16/60
4233/4233 - 0s - loss: 0.3055 - accuracy: 0.9017
Epoch 17/60

In [142]:
#Quantifying the model

model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1411/1411 - 0s - loss: 0.2942 - accuracy: 0.9093
Normal Neural Network - Loss: 0.29417793117965224, Accuracy: 0.9092841744422913


In [174]:
#Make some predictions
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [175]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['negative' 'negative' 'negative' 'negative' 'negative']
Actual Labels: ['negative', 'negative', 'negative', 'negative', 'negative']


In [145]:
#Save model
model.save('tensorflow_model.pkl')

INFO:tensorflow:Assets written to: tensorflow_model.pkl/assets


In [154]:
#import model and make prediction
import tensorflow as tf

In [155]:
model_tensor = tf.keras.models.load_model("tensorflow_model.pkl")

In [193]:
prediction=model_tensor.predict_classes([[ 19, -1.288428, -0.906829, 0.567652, 0.694287, -0.835508, 0.578024, 0.541564, -0.295726, -0.325903]])

In [194]:
print(prediction)

[0]


In [195]:
print(model_tensor.predict_classes([[ 19, -1.288428, -0.906829, 0.567652, 0.694287, -0.835508, 0.578024, 0.541564, -0.295726, -0.325903]]))

[0]
